In [13]:
# -*- coding: utf-8 -*-
# 缺值統計（每欄位）— 讀檔、偵測占位符、產出彙整表與CSV

import pandas as pd
import numpy as np
from collections import Counter



In [16]:
CSV_PATH = "adult_data_no_duplicates.csv"
# 1) 先「不把任何東西當成NA」讀入，以便找出原始的占位符
raw = pd.read_csv(
    CSV_PATH,
    dtype=str,
    keep_default_na=False,  # 不自動當NA
    na_values=[],           # 不額外指定NA
    encoding_errors="ignore"
)

# 常見缺值占位符（可依需要自行增減）
PLACEHOLDERS = {"?", "NA", "N/A", "na", "null", "None", "\\N"}

def is_blank(x) -> bool:
    """判斷是否空字串或只含空白字元。"""
    if x is None:
        return True
    if isinstance(x, float) and np.isnan(x):
        return True
    if isinstance(x, str):
        return len(x.strip()) == 0
    return False

# 統計各欄位在「原始讀取」時所出現的占位符類型與次數
placeholder_counts = {col: Counter() for col in raw.columns}
for col in raw.columns:
    for val in raw[col].astype(str):
        if is_blank(val):
            placeholder_counts[col]["<blank/whitespace>"] += 1
        elif val in PLACEHOLDERS:
            placeholder_counts[col][val] += 1

# 2) 再用「將占位符視為NA」的方式重讀，統一缺值
NA_VALUES = ["", " ", "  ", "   ", "\t", "\n", "?", "NA", "N/A", "na", "null", "None", "\\N"]
df = pd.read_csv(
    CSV_PATH,
    dtype=str,
    keep_default_na=True,   # 啟用pandas預設NA規則
    na_values=NA_VALUES,    # 再加入我們客製的占位符
    encoding_errors="ignore"
)

# 3) 產生每欄位缺值統計表
total_rows = len(df)
summary_rows = []

for col in df.columns:
    missing_count = df[col].isna().sum()
    non_missing_count = total_rows - missing_count
    missing_percent = (missing_count / total_rows * 100.0) if total_rows else 0.0

    # 列出該欄位最常見的占位符（最多3個）
    ph = placeholder_counts[col]
    top_ph_tokens = ", ".join([f"{k}:{ph[k]}" for k, _ in ph.most_common(3)]) if ph else ""

    # 粗略推斷欄位型態（>95% 可轉換為數字 -> numeric-like）
    col_series = df[col].dropna()
    numeric_ratio = pd.to_numeric(col_series, errors="coerce").notna().mean() if len(col_series) else np.nan
    inferred_type = "numeric-like" if pd.notna(numeric_ratio) and numeric_ratio > 0.95 else "string-like"

    summary_rows.append({
        "column": col,
        "missing_count": int(missing_count),
        "missing_percent": round(missing_percent, 2),
        "non_missing_count": int(non_missing_count),
        "inferred_type": inferred_type,
        "placeholder_tokens_detected_in_raw": top_ph_tokens
    })

summary = (
    pd.DataFrame(summary_rows)
      .sort_values(by=["missing_count", "column"], ascending=[False, True])
      .reset_index(drop=True)
)

# 4) 顯示與輸出
display(summary)                     # 在 Notebook 內顯示互動表格

# ===（選用）每列缺值數量，可視化/過濾時會有用===
df["__row_missing_count__"] = df.isna().sum(axis=1)
display(df["__row_missing_count__"].value_counts().sort_index())

,column,missing_count,missing_percent,non_missing_count,inferred_type,placeholder_tokens_detected_in_raw
0,occupation,1843,5.66,30694,string-like,?:1843
1,workclass,1836,5.64,30701,string-like,?:1836
2,native-country,582,1.79,31955,string-like,?:582
3,age,0,0.00,32537,numeric-like,
4,capital-gain,0,0.00,32537,numeric-like,
5,capital-loss,0,0.00,32537,numeric-like,
6,class,0,0.00,32537,string-like,
7,education,0,0.00,32537,string-like,
8,education-num,0,0.00,32537,numeric-like,
9,fnlwgt,0,0.00,32537,numeric-like,


__row_missing_count__
0    30139
1      562
2     1809
3       27
Name: count, dtype: int64

In [17]:
raw = pd.read_csv(CSV_PATH, dtype=str, keep_default_na=False, na_values=[])

# 1) 總列數/欄數
len(raw), raw.shape[1]

# 2) 數 '?'：每欄與每列
per_col_q = (raw == "?").sum(axis=0).sort_values(ascending=False)
rows_with_any_q = (raw == "?").any(axis=1).sum()

per_col_q, rows_with_any_q

# 3)（可選）把 '?' 統一視為缺值，再看每列是否有缺值
NA_VALUES = ["", " ", "\t", "\n", "?", "NA", "N/A", "na", "null", "None", "\\N"]
df = pd.read_csv(CSV_PATH, dtype=str, keep_default_na=True, na_values=NA_VALUES)
rows_with_any_na = df.isna().any(axis=1).sum()
rows_with_any_na  # 這裡應該也會是 2398

np.int64(2398)

In [18]:
raw = pd.read_csv(CSV_PATH, dtype=str, keep_default_na=False, na_values=[], encoding_errors="ignore")

# (A) 缺失值出現在哪些屬性：每欄位 '?' 次數與比例
per_attr_q = (raw == "?").sum().sort_values(ascending=False).rename("q_count").to_frame()
per_attr_q["q_percent"] = per_attr_q["q_count"] / len(raw) * 100
display(per_attr_q)

# (B) 每列含幾個 '?' 的分佈（0/1/2/3...）
q_per_row = (raw == "?").sum(axis=1)
dist = q_per_row.value_counts().sort_index().rename("row_count").to_frame()
dist.index.name = "num_q_in_row"
display(dist)

# (C) 篩出同列中同時有 2 或 3 個 '?' 的資料，並標註哪些欄位是 '?'
mask_q = raw.eq("?")
def with_q_columns(df_subset, mask_subset):
    cols = list(mask_subset.columns)
    qcols = mask_subset.apply(lambda r: [c for c, v in zip(cols, r) if bool(v)], axis=1)
    out = df_subset.copy()
    out.insert(0, "__q_count__", mask_subset.sum(axis=1).values)
    out.insert(1, "__q_columns__", qcols.values)
    return out

rows2 = with_q_columns(raw[q_per_row == 2], mask_q[q_per_row == 2])
rows3 = with_q_columns(raw[q_per_row == 3], mask_q[q_per_row == 3])

display(rows2)
display(rows3)



,q_count,q_percent
occupation,1843,5.664321
workclass,1836,5.642807
native-country,582,1.788733
fnlwgt,0,0.000000
education,0,0.000000
education-num,0,0.000000
age,0,0.000000
marital-status,0,0.000000
relationship,0,0.000000
sex,0,0.000000


,row_count
num_q_in_row,
0,30139
1,562
2,1809
3,27


,__q_count__,__q_columns__,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
27,2,"[workclass, occupation]",54,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,60,South,>50K
69,2,"[workclass, occupation]",25,?,200681,Some-college,10,Never-married,?,Own-child,White,Male,0,0,40,United-States,<=50K
77,2,"[workclass, occupation]",67,?,212759,10th,6,Married-civ-spouse,?,Husband,White,Male,0,0,2,United-States,<=50K
106,2,"[workclass, occupation]",17,?,304873,10th,6,Never-married,?,Own-child,White,Female,34095,0,32,United-States,<=50K
128,2,"[workclass, occupation]",35,?,129305,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32506,2,"[workclass, occupation]",35,?,320084,Bachelors,13,Married-civ-spouse,?,Wife,White,Female,0,0,55,United-States,>50K
32507,2,"[workclass, occupation]",30,?,33811,Bachelors,13,Never-married,?,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
32515,2,"[workclass, occupation]",71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,White,Male,0,0,10,United-States,>50K
32517,2,"[workclass, occupation]",41,?,202822,HS-grad,9,Separated,?,Not-in-family,Black,Female,0,0,32,United-States,<=50K


,__q_count__,__q_columns__,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
61,3,"[workclass, occupation, native-country]",32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,White,Male,0,0,40,?,<=50K
297,3,"[workclass, occupation, native-country]",39,?,157443,Masters,14,Married-civ-spouse,?,Wife,Asian-Pac-Islander,Female,3464,0,40,?,<=50K
1152,3,"[workclass, occupation, native-country]",24,?,35633,Some-college,10,Never-married,?,Not-in-family,White,Male,0,0,40,?,<=50K
1676,3,"[workclass, occupation, native-country]",64,?,168340,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,40,?,>50K
2513,3,"[workclass, occupation, native-country]",47,?,174525,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,3942,0,40,?,<=50K
3131,3,"[workclass, occupation, native-country]",25,?,237865,Some-college,10,Never-married,?,Own-child,Black,Male,0,0,40,?,<=50K
3579,3,"[workclass, occupation, native-country]",21,?,180303,Bachelors,13,Never-married,?,Not-in-family,Asian-Pac-Islander,Male,0,0,25,?,<=50K
3834,3,"[workclass, occupation, native-country]",32,?,169886,Bachelors,13,Never-married,?,Not-in-family,White,Female,0,0,20,?,<=50K
6057,3,"[workclass, occupation, native-country]",35,?,163582,10th,6,Divorced,?,Unmarried,White,Female,0,0,16,?,<=50K
7860,3,"[workclass, occupation, native-country]",29,?,125159,Some-college,10,Never-married,?,Not-in-family,Black,Male,0,0,36,?,<=50K


In [17]:
# -*- coding: utf-8 -*-
# 顯示「更多筆」補值前/後對照 + 只列出有變動欄位（長表）

import pandas as pd
import numpy as np
from IPython.display import display

# === 參數 ===
CSV_PATH  = "adult_test_no_duplicates.csv"  # ← 換成你的檔案路徑
SHOW_N    = 20        # 要顯示幾筆（可改大/小）
SHOW_MODE = "random"  # "head" | "random" （顯示前幾筆 或 隨機抽樣）

NA_VALUES = ["", " ", "  ", "\t", "\n", "?", "NA", "N/A", "na", "null", "None", "\\N"]
NUMERIC_COLS = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
CAT_WITH_MISSING = ['workclass','occupation','native-country']

# === 讀檔，將占位符視為缺值 ===
df_before = pd.read_csv(CSV_PATH, dtype=str, keep_default_na=True, na_values=NA_VALUES, encoding_errors="ignore")

# 數值欄位轉數值
for c in NUMERIC_COLS:
    if c in df_before.columns:
        df_before[c] = pd.to_numeric(df_before[c], errors='coerce')

# === 進階補值（類別：群組眾數 + Unknown；數值：中位數） ===
df_after = df_before.copy()

def impute_by_group_mode(df, target, group_cols, fallback="Unknown"):
    modes = df.groupby(group_cols)[target].transform(
        lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan
    )
    df[target] = df[target].fillna(modes)
    df[target] = df[target].fillna(fallback)
    return df

# workclass：以 (education, marital-status) 分群 → 眾數 → Unknown
if set(['workclass','education','marital-status']).issubset(df_after.columns):
    df_after = impute_by_group_mode(df_after, 'workclass', ['education','marital-status'], fallback="Unknown")

# occupation：以 (workclass, education, sex) 分群 → 眾數 → Unknown
if set(['occupation','workclass','education','sex']).issubset(df_after.columns):
    df_after = impute_by_group_mode(df_after, 'occupation', ['workclass','education','sex'], fallback="Unknown")

# native-country：直接 Unknown（避免偏誤）
if 'native-country' in df_after.columns:
    df_after['native-country'] = df_after['native-country'].fillna('Unknown')

# 數值欄位：中位數補值
for c in NUMERIC_COLS:
    if c in df_after.columns and df_after[c].isna().any():
        df_after[c] = df_after[c].fillna(df_after[c].median())

# === 選要比較的欄位 ===
compare_cols = [c for c in CAT_WITH_MISSING + NUMERIC_COLS if c in df_before.columns]

# 只挑「原本在 compare_cols 有缺值」的列
mask_missing_any = df_before[compare_cols].isna().any(axis=1)
candidates = df_before[mask_missing_any].index

if len(candidates) == 0:
    print("目前資料在 compare_cols 中沒有缺值，請確認欄位或換其他欄位。")
else:
    if SHOW_MODE == "head":
        idxs = candidates[:SHOW_N]
    else:
        # random
        if len(candidates) <= SHOW_N:
            idxs = candidates
        else:
            idxs = pd.Index(np.random.RandomState(42).choice(candidates, size=SHOW_N, replace=False))

    # --- 並排對照（寬表） ---
    before_view = df_before.loc[idxs, compare_cols].copy().add_suffix("_before")
    after_view  = df_after.loc[idxs,  compare_cols].copy().add_suffix("_after")
    side_by_side = before_view.join(after_view)

    # 讓相同欄位的 before/after 交錯排列，閱讀更直覺
    ordered_cols = []
    for c in compare_cols:
        b, a = f"{c}_before", f"{c}_after"
        if b in side_by_side.columns and a in side_by_side.columns:
            ordered_cols += [b, a]
    side_by_side = side_by_side[ordered_cols]

    print(f"=== 補值前後（{SHOW_MODE} 顯示 {len(idxs)} 筆）===")
    display(side_by_side)

    # --- 只列出「有變動的欄位」的長表 ---
    rows = []
    for i in idxs:
        for c in compare_cols:
            bv = df_before.at[i, c]
            av = df_after.at[i, c]
            # 統一成字串便於比較
            bv_str = "NaN" if pd.isna(bv) else str(bv)
            av_str = "NaN" if pd.isna(av) else str(av)
            if bv_str != av_str:
                rows.append({"row_index": i, "column": c, "before": bv, "after": av})

    diff_long = pd.DataFrame(rows, columns=["row_index","column","before","after"])
    print("=== 僅顯示『值有變動』的欄位（長表） ===")
    display(diff_long if not diff_long.empty else pd.DataFrame(columns=["row_index","column","before","after"]))

# （可選）檢查整體缺值：補值前 vs 補值後
def missing_summary(d):
    s = d.isna().sum().sort_values(ascending=False)
    return pd.DataFrame({"missing_count": s, "missing_percent": s/len(d)*100})

print("=== 缺值統計：補值前 ===")
display(missing_summary(df_before))
print("=== 缺值統計：補值後 ===")
display(missing_summary(df_after))

=== 補值前後（random 顯示 20 筆）===


,workclass_before,workclass_after,occupation_before,occupation_after,native-country_before,native-country_after,age_before,age_after,fnlwgt_before,fnlwgt_after,education-num_before,education-num_after,capital-gain_before,capital-gain_after,capital-loss_before,capital-loss_after,hours-per-week_before,hours-per-week_after
5319,NaN,Private,NaN,Other-service,United-States,United-States,24,24,267955,267955,5,5,0,0,0,0,20,20
2464,NaN,Private,NaN,Craft-repair,United-States,United-States,69,69,183958,183958,9,9,0,0,0,0,8,8
20586,NaN,Private,NaN,Adm-clerical,United-States,United-States,54,54,155755,155755,9,9,4416,4416,0,0,25,25
17628,NaN,Private,NaN,Adm-clerical,United-States,United-States,41,41,29115,29115,10,10,0,0,0,0,20,20
17100,Private,Private,Prof-specialty,Prof-specialty,NaN,Unknown,25,25,164488,164488,13,13,0,0,0,0,40,40
3902,NaN,Private,NaN,Craft-repair,United-States,United-States,22,22,131230,131230,10,10,0,0,0,0,40,40
32312,NaN,Private,NaN,Other-service,United-States,United-States,18,18,156608,156608,7,7,0,0,0,0,25,25
22771,NaN,Private,NaN,Craft-repair,United-States,United-States,27,27,204074,204074,9,9,0,0,0,0,40,40
18973,Private,Private,Machine-op-inspct,Machine-op-inspct,NaN,Unknown,44,44,237993,237993,15,15,0,0,0,0,45,45
19342,Private,Private,Exec-managerial,Exec-managerial,NaN,Unknown,33,33,35309,35309,13,13,0,0,0,0,40,40


=== 僅顯示『值有變動』的欄位（長表） ===


,row_index,column,before,after
0,5319,workclass,NaN,Private
1,5319,occupation,NaN,Other-service
2,2464,workclass,NaN,Private
3,2464,occupation,NaN,Craft-repair
4,20586,workclass,NaN,Private
5,20586,occupation,NaN,Adm-clerical
6,17628,workclass,NaN,Private
7,17628,occupation,NaN,Adm-clerical
8,17100,native-country,NaN,Unknown
9,3902,workclass,NaN,Private


=== 缺值統計：補值前 ===


,missing_count,missing_percent
occupation,1843,5.664321
workclass,1836,5.642807
native-country,582,1.788733
fnlwgt,0,0.000000
education,0,0.000000
education-num,0,0.000000
age,0,0.000000
marital-status,0,0.000000
relationship,0,0.000000
sex,0,0.000000


=== 缺值統計：補值後 ===


,missing_count,missing_percent
age,0,0.0
workclass,0,0.0
fnlwgt,0,0.0
education,0,0.0
education-num,0,0.0
marital-status,0,0.0
occupation,0,0.0
relationship,0,0.0
race,0,0.0
sex,0,0.0


In [25]:
NUMERIC_COLS = ['age','fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
CAT_WITH_MISSING = ['workclass','occupation','native-country']

# === 讀檔（補值前狀態） ===
df_before = pd.read_csv(CSV_PATH, dtype=str, keep_default_na=True, na_values=NA_VALUES, encoding_errors="ignore")
for c in NUMERIC_COLS:
    if c in df_before.columns:
        df_before[c] = pd.to_numeric(df_before[c], errors='coerce')

# === 進階補值（類別：群組眾數 + Unknown；數值：中位數） ===
df_after = df_before.copy()

def impute_by_group_mode(df, target, group_cols, fallback="Unknown"):
    modes = df.groupby(group_cols)[target].transform(
        lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan
    )
    df[target] = df[target].fillna(modes)
    df[target] = df[target].fillna(fallback)
    return df

if set(['workclass','education','marital-status']).issubset(df_after.columns):
    df_after = impute_by_group_mode(df_after, 'workclass', ['education','marital-status'])
if set(['occupation','workclass','education','sex']).issubset(df_after.columns):
    df_after = impute_by_group_mode(df_after, 'occupation', ['workclass','education','sex'])
if 'native-country' in df_after.columns:
    df_after['native-country'] = df_after['native-country'].fillna('Unknown')

numeric_medians = {}
for c in NUMERIC_COLS:
    if c in df_after.columns:
        m = df_after[c].median()  # 即使沒有缺也記錄中位數
        numeric_medians[c] = m
        if df_after[c].isna().any():
            df_after[c] = df_after[c].fillna(m)

# === 1) 類別欄位：補進去的類別與筆數 ===
cat_results = []
for col in [c for c in CAT_WITH_MISSING if c in df_before.columns]:
    mask_missing_before = df_before[col].isna()
    if mask_missing_before.any():
        filled_values = df_after.loc[mask_missing_before, col]
        counts = filled_values.value_counts(dropna=False)
        for val, cnt in counts.items():
            cat_results.append({
                "column": col,
                "imputed_value": "NaN" if pd.isna(val) else str(val),
                "count": int(cnt)
            })

cat_table = pd.DataFrame(cat_results, columns=["column","imputed_value","count"])
if not cat_table.empty:
    cat_table = cat_table.sort_values(by=["column","count"], ascending=[True, False]).reset_index(drop=True)

print("=== 類別欄位：所有被補進去的『類別』與其筆數 ===")
display(cat_table if not cat_table.empty else pd.DataFrame(columns=["column","imputed_value","count"]))

# === 2) 數值欄位：補進去的數值與筆數 ===
num_results = []
for col in [c for c in NUMERIC_COLS if c in df_before.columns]:
    mask_missing_before = df_before[col].isna()
    if mask_missing_before.any():
        filled_values = df_after.loc[mask_missing_before, col]
        counts = filled_values.value_counts(dropna=False)
        for val, cnt in counts.items():
            # 轉成 float（轉不動維持 NaN）
            try:
                v = float(val) if pd.notna(val) else np.nan
            except Exception:
                v = np.nan
            num_results.append({
                "column": col,
                "imputed_value": v,
                "count": int(cnt)
            })

num_table = pd.DataFrame(num_results, columns=["column","imputed_value","count"])
if not num_table.empty:
    num_table = num_table.sort_values(by=["column","count"], ascending=[True, False]).reset_index(drop=True)

print("=== 數值欄位：所有被補進去的『數值』與其筆數 ===")
display(num_table if not num_table.empty else pd.DataFrame(columns=["column","imputed_value","count"]))

# === 3) 每個數值欄位使用的中位數 ===
median_table = pd.DataFrame(
    [{"column": col, "median_used": float(m)} for col, m in numeric_medians.items()]
).sort_values("column").reset_index(drop=True)

print("=== 數值欄位：本次補值所使用的中位數（median） ===")
display(median_table)

# === 4)（可選）每欄位被補的列數（補值前有缺的筆數） ===
summary_rows = []
all_cols = set(CAT_WITH_MISSING + NUMERIC_COLS)
for col in [c for c in df_before.columns if c in all_cols]:
    summary_rows.append({"column": col, "imputed_row_count": int(df_before[col].isna().sum())})
summary_table = pd.DataFrame(summary_rows, columns=["column","imputed_row_count"]).sort_values("column").reset_index(drop=True)
print("=== 每欄位：被補的列數總計 ===")
display(summary_table)

=== 類別欄位：所有被補進去的『類別』與其筆數 ===


,column,imputed_value,count
0,native-country,Unknown,582
1,occupation,Craft-repair,807
2,occupation,Adm-clerical,571
3,occupation,Other-service,191
4,occupation,Exec-managerial,137
5,occupation,Prof-specialty,117
6,occupation,Machine-op-inspct,10
7,occupation,Unknown,7
8,occupation,Farming-fishing,3
9,workclass,Private,1833


=== 數值欄位：所有被補進去的『數值』與其筆數 ===


,column,imputed_value,count


=== 數值欄位：本次補值所使用的中位數（median） ===


,column,median_used
0,age,37.0
1,capital-gain,0.0
2,capital-loss,0.0
3,education-num,10.0
4,fnlwgt,178356.0
5,hours-per-week,40.0


=== 每欄位：被補的列數總計 ===


,column,imputed_row_count
0,age,0
1,capital-gain,0
2,capital-loss,0
3,education-num,0
4,fnlwgt,0
5,hours-per-week,0
6,native-country,582
7,occupation,1843
8,workclass,1836
